In [31]:
import pandas as pd
from web3 import Web3
import json
import time

# Load IoT dataset generated from Homework 1
df = pd.read_csv("iot_data.csv")
df.head()

timestamp package_id  order_date delivery_date  \
0  2025-05-24 21:55:38     PKG001  2025-05-22    2025-06-01   
1  2025-05-25 18:12:38     PKG002  2025-05-20    2025-05-27   
2  2025-05-24 21:59:38     PKG003  2025-05-19    2025-05-28   
3  2025-05-25 17:42:38     PKG004  2025-05-22    2025-06-01   
4  2025-05-25 18:32:38     PKG005  2025-05-21    2025-05-30   

                           origin     origin_coordinates  \
0        Puerto Princesa, Palawan   9.7808779,118.730191   
1  General Santos, South Cotabato  6.1122217,125.1721893   
2        Tacurong, Sultan Kudarat  6.6884368,124.6786576   
3   San Carlos, Negros Occidental  10.486048,123.4190385   
4      Ozamiz, Misamis Occidental  8.1470175,123.8459793   

                  current_location     current_coordinates  \
0                       Tubo, Abra     9.777611,118.740863   
1  Pagalungan, Maguindanao del Sur     6.113452,125.166222   
2                 Tubungan, Iloilo   10.7653357,122.318261   
3           Nampicuan, Nueva Ecija  15.7321724,120.6386089   
4                 Malvar, Batangas     8.140163,123.865499   

                 delivery_location    delivery_coordinates perishable  \
0                       Tubo, Abra  17.2582154,120.7248438         No   
1  Pagalungan, Maguindanao del Sur   7.0599392,124.6992377        Yes   
2                 Tubungan, Iloilo   10.7653357,122.318261        Yes   
3           Nampicuan, Nueva Ecija  15.7321724,120.6386089         No   
4                 Malvar, Batangas   14.0472848,121.157631        Yes   

   temperature_celsius temperature_issue      status  
0                21.20               NaN  In Transit  
1                11.84   Temp Alert >7°C  In Transit  
2                10.41   Temp Alert >7°C   Delivered  
3                20.66               NaN   Delivered  
4                 5.41            Normal  In Transit

In [32]:
# Connect to local Ganache blockchain
ganache_url = "http://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))

if web3.is_connected():
    print("Connected to Ganache successfully.")
else:
    raise ConnectionError("Connection failed. Ensure Ganache is running.")

Connected to Ganache successfully.


In [33]:
# Load the ABI and contract address from Remix
with open("abi.json", "r") as abi_file:
    abi = json.load(abi_file)

contract_address = web3.to_checksum_address("0x14208d20d96abB5b79136dF68683550b1CE64CB8")
contract = web3.eth.contract(address=contract_address, abi=abi)

In [34]:
# Set default sender (typically the first account from Ganache)
web3.eth.default_account = web3.eth.accounts[0]
print(f"Connected to Smart Contract at {contract_address}")

Connected to Smart Contract at 0x14208d20d96abB5b79136dF68683550b1CE64CB8


In [35]:
# Define a function to send each row of IoT data to the blockchain
def send_iot_data(row):
    txn = contract.functions.storeData(
        str(row["package_id"]),
        str(row["order_date"]),
        str(row["delivery_date"]),
        str(row["origin"]),
        str(row["current_location"]),
        str(row["delivery_location"]),
        str(row["perishable"]),
        str(row["temperature_celsius"]),
        str(row["temperature_issue"]),
        str(row["status"])
    ).transact({
        'from': web3.eth.default_account,
        'gas': 3000000
    })
    receipt = web3.eth.wait_for_transaction_receipt(txn)
    print(f"Stored {row['package_id']} | Status: {row['status']} | Txn Hash: {receipt.transactionHash.hex()}")

In [36]:
# Loop through the DataFrame and send each row to the blockchain
for _, row in df.iterrows():
    send_iot_data(row)
    time.sleep(1)  # Avoid flooding the blockchain with transactions

Stored PKG001 | Status: In Transit | Txn Hash: 95f66e8c7957b8d6239362a5d3cd904bb82f4424b4d516ea3dfffc6715cadfef
Stored PKG002 | Status: In Transit | Txn Hash: 513bcdb7820da2367c9345c2a094ae6716becb2fc973a429d06d0223a6e9ec21
Stored PKG003 | Status: Delivered | Txn Hash: 38065fa452b6879f32c5f7f1f961028e7042f88051e06da2b9a22770370d4f4b
Stored PKG004 | Status: Delivered | Txn Hash: 156c47afa08def083a48a00e1215922f7c1c359e0174d34ec6b9e5d9472c17a2
Stored PKG005 | Status: In Transit | Txn Hash: 8eae565accf7028ac7caf3a814f8049bccfd9b4bef417bc6534cc349adedf2d3
Stored PKG006 | Status: Delayed | Txn Hash: d5b7b743c8428fc53adbbd5fe0a04e9e2943d04c0c7af8087a8f9454c65aedf2
Stored PKG007 | Status: Awaiting Pickup | Txn Hash: a98f3dfc78c84f4c1395f7e53b0274a9392f42c61e962b7a7e6330b40b4d3bea
Stored PKG008 | Status: In Transit | Txn Hash: ca6bc0a7fd1e54ef38ce44ef267ea21748bc1f237027192ed4c204909fe7f238
Stored PKG009 | Status: Delivered | Txn Hash: fb7c90b3202409d0c1ba1745d3c9240b020690d039962f78e346f93355

In [37]:
# Verify the total number of records stored
total_records = contract.functions.getTotalRecords().call()
print(f"Total IoT records stored: {total_records}")

Total IoT records stored: 100


In [38]:
# Retrieve and display the first stored record
first_record = contract.functions.getRecord(0).call()
print("First Stored Record:", first_record)

First Stored Record: [1748174030, 'PKG001', '2025-05-22', '2025-06-01', 'Puerto Princesa, Palawan', 'Tubo, Abra', 'Tubo, Abra', 'No', '21.2', 'nan', 'In Transit']
